In [2]:
import os
import json
import requests
folder = "./parts_index"
filenames = os.listdir(folder)

print(filenames)

['Plate 1 x 10 (4477).png', 'Plate 1 x 12 (60479).png', 'Plate 1x1 (3024).png', 'Plate 1x2 (3023).png', 'Plate 1x3 (3623).png', 'Plate 1x4(3710).png', 'Plate 1x5(78329).png', 'Plate 1x6(3666).png', 'Plate 1x8(3460).png', 'Plate 2 x 10(3832).png', 'Plate 2 x 12(2445).png', 'Plate 2 x 14(91988).png', 'Plate 2 x 16(4282).png', 'Plate 2 x 2 (3022).png', 'Plate 2 x 2 Corner (2420).png', 'Plate 2 x 3 (3021).png', 'Plate 2 x 4(3020).png', 'Plate 2 x 6 (3795).png', 'Plate 2 x 8(3034).png', 'Plate 3 x 3 Corner(77844).png', 'Plate 3 x 3(11212).png', 'Plate 4 x 10 (3030).png', 'Plate 4 x 12 (3029).png', 'Plate 4 x 4 (3031).png', 'Plate 4 x 4 Corner (2639).png', 'Plate 4 x 6 (3032).png', 'Plate 4 x 8 (3035).png', 'Plate 6 x 10 (3033).png', 'Plate 6 x 12 (3028).png', 'Plate 6 x 14 (3456).png', 'Plate 6 x 16 (3027).png', 'Plate 6 x 24 (3026).png', 'Plate 6 x 6 (3958).png', 'Plate 6 x 8 (3036).png', 'Plate 8 x 16(92438).png', 'Plate 8 x 8 (42534).png']


In [15]:
extras_url = "https://mecabricks.com/assets/parts/v2/shared/extras.json"
extras = requests.get(extras_url).json()
filename = "extras.json"
with open(filename, "w") as f:
    json.dump(extras, f, indent=4)    

In [5]:
parts = []
for filename in filenames:
    # filenames are of the form "<name> (<id>).<ext>"
    name, rest = filename.split("(")
    id, rest = rest.split(")")
    parts.append({
        "name": name.strip(" ()"),
        "id": id.strip(" ()"),
    })

print(parts)

for part in parts:
    print(f"Getting geometry for {part['name']} ({part['id']})")
    geometry_url = f"https://mecabricks.com/assets/parts/v2/geometries/{part['id']}.json"

    geometry_response = requests.get(geometry_url)
    part_geometry = geometry_response.json()
    if "error" in part_geometry:
        print(f"Error: {part_geometry['error']}")
    # elif ["vertices", "faces", "uvs", "normals"] not in part_geometry:
    elif "vertices" not in part_geometry or "faces" not in part_geometry or "uvs" not in part_geometry or "normals" not in part_geometry:
        print(f"Error: Invalid geometry for {part['name']} ({part['id']})")
    else:
        print(f"Got geometry for {part['name']} ({part['id']})")
        part["geometry"] = part_geometry

    config_url = f"https://mecabricks.com/assets/parts/v2/configs/{part['id']}.json"
    config_response = requests.get(config_url)
    part_config = config_response.json()
    if "error" in part_config:
        print(f"Error: {part_config['error']}")
    else:
        print(f"Got config for {part['name']} ({part['id']})")
        part["config"] = part_config

# save each part as a json file
for part in parts:
    filename = f"./parts_json/{part['name']} ({part['id']}).json"
    with open(filename, "w") as f:
        json.dump(part, f)

[{'name': 'Plate 1 x 10', 'id': '4477'}, {'name': 'Plate 1 x 12', 'id': '60479'}, {'name': 'Plate 1x1', 'id': '3024'}, {'name': 'Plate 1x2', 'id': '3023'}, {'name': 'Plate 1x3', 'id': '3623'}, {'name': 'Plate 1x4', 'id': '3710'}, {'name': 'Plate 1x5', 'id': '78329'}, {'name': 'Plate 1x6', 'id': '3666'}, {'name': 'Plate 1x8', 'id': '3460'}, {'name': 'Plate 2 x 10', 'id': '3832'}, {'name': 'Plate 2 x 12', 'id': '2445'}, {'name': 'Plate 2 x 14', 'id': '91988'}, {'name': 'Plate 2 x 16', 'id': '4282'}, {'name': 'Plate 2 x 2', 'id': '3022'}, {'name': 'Plate 2 x 2 Corner', 'id': '2420'}, {'name': 'Plate 2 x 3', 'id': '3021'}, {'name': 'Plate 2 x 4', 'id': '3020'}, {'name': 'Plate 2 x 6', 'id': '3795'}, {'name': 'Plate 2 x 8', 'id': '3034'}, {'name': 'Plate 3 x 3 Corner', 'id': '77844'}, {'name': 'Plate 3 x 3', 'id': '11212'}, {'name': 'Plate 4 x 10', 'id': '3030'}, {'name': 'Plate 4 x 12', 'id': '3029'}, {'name': 'Plate 4 x 4', 'id': '3031'}, {'name': 'Plate 4 x 4 Corner', 'id': '2639'}, {'na

In [3]:
extras = json.load(open("./extras.json"))

In [26]:
def get_vertices_from_list(vertices_list):
    vertices = []
    for i in range(0, len(vertices_list), 3):
        vertices.append((vertices_list[i], vertices_list[i+1], vertices_list[i+2]))
    return vertices

def get_faces_from_list(faces_list, jump=14):
    faces = []
    i = 0
    while i < len(faces_list):
        # write quad as two triangles
        faces.append((faces_list[i+1], faces_list[i+2], faces_list[i+3]))
        faces.append((faces_list[i+1], faces_list[i+3], faces_list[i+4]))

        i += jump

    return faces

def combine_objects(vertices1, faces1, position1, vertices2, faces2, position2):
    # Update vertices1 in place with position1
    vertices1 = [(v[0] + position1[0], v[1] + position1[1], v[2] + position1[2]) for v in vertices1]
    
    # Update vertices2 in place with position2
    vertices2 = [(v[0] + position2[0], v[1] + position2[1], v[2] + position2[2]) for v in vertices2]

    # Offset indices in faces2 by the number of vertices in vertices1
    vertex_offset = len(vertices1)
    faces2 = [[vertex_index + vertex_offset for vertex_index in face] for face in faces2]

    # Return the combined vertices and faces
    return vertices1 + vertices2, faces1 + faces2

def get_extra_geometry(part):
    config = part["config"]
    vertices = []
    faces = []
    knobs =  config["geometry"]["extras"]["knobs"]
    for knob in knobs:
        knob_geometry = extras["knobs"][str(knob["type"])]

        # Add position to each vertex
        vertices_list = knob_geometry["vertices"]
        new_vertices = get_vertices_from_list(vertices_list)

        faces_list = knob_geometry["faces"]
        new_faces = get_faces_from_list(faces_list)

        vertices, faces = combine_objects(vertices, faces, (0, 0, 0), new_vertices, new_faces, knob["transform"]["position"])

    return vertices, faces

def get_main_geometry(part):
    geometry = part["geometry"]
    config = part["config"]

    # Get vertices
    vertices_list = geometry["vertices"]
    vertices = get_vertices_from_list(vertices_list)
        
    # Get faces
    faces_list = geometry["faces"]
    decoration_uvs_len = len(config["decorationUvs"])
    jump = 13 + 4*decoration_uvs_len
    faces = get_faces_from_list(faces_list, jump)

    return vertices, faces

def get_obj_lines_from_json(json_data):
    obj_lines = [
        "# OBJ file generated by json_to_obj.py",
        "o part",
    ]

    # Get main geometry
    main_vertices, main_faces = get_main_geometry(json_data)

    # Get extra geometry
    extra_vertices, extra_faces = get_extra_geometry(json_data)

    all_vertices, all_faces = combine_objects(main_vertices, main_faces, (0, 0, 0), extra_vertices, extra_faces, (0, 0, 0))

    for vertex in all_vertices:
        obj_lines.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}")
    
    for face in all_faces:
        obj_lines.append(f"f {face[0]+1} {face[1]+1} {face[2]+1}")

    return obj_lines
    

# now we want to save each part as an obj file
for part in parts:
    print(f"Saving {part['name']} ({part['id']}) as obj")
    filename = f"./parts_obj/{part['name']} ({part['id']}).obj"
    with open(filename, "w") as f:
        obj_str = "\n".join(get_obj_lines_from_json(part))
        f.write(obj_str)


Saving Plate 1 x 10 (4477) as obj
Saving Plate 1 x 12 (60479) as obj
Saving Plate 1x1 (3024) as obj
Saving Plate 1x2 (3023) as obj
Saving Plate 1x3 (3623) as obj
Saving Plate 1x4 (3710) as obj
Saving Plate 1x5 (78329) as obj
Saving Plate 1x6 (3666) as obj
Saving Plate 1x8 (3460) as obj
Saving Plate 2 x 10 (3832) as obj
Saving Plate 2 x 12 (2445) as obj
Saving Plate 2 x 14 (91988) as obj
Saving Plate 2 x 16 (4282) as obj
Saving Plate 2 x 2 (3022) as obj
Saving Plate 2 x 2 Corner (2420) as obj
Saving Plate 2 x 3 (3021) as obj
Saving Plate 2 x 4 (3020) as obj
Saving Plate 2 x 6 (3795) as obj
Saving Plate 2 x 8 (3034) as obj
Saving Plate 3 x 3 Corner (77844) as obj
Saving Plate 3 x 3 (11212) as obj
Saving Plate 4 x 10 (3030) as obj
Saving Plate 4 x 12 (3029) as obj
Saving Plate 4 x 4 (3031) as obj
Saving Plate 4 x 4 Corner (2639) as obj
Saving Plate 4 x 6 (3032) as obj
Saving Plate 4 x 8 (3035) as obj
Saving Plate 6 x 10 (3033) as obj
Saving Plate 6 x 12 (3028) as obj
Saving Plate 6 x 14 (